# Week 12 — Capstone (캠페인/가격/채널 전략 미니 프로젝트)

이 노트북은 **실전 통계 → 해석 → 비즈니스 액션**을 훈련하기 위한 템플릿입니다.
각 섹션의 ✅ 체크포인트를 채우면서 진행하세요.


## 1) Business Questions
- 캠페인 성과를 진단하고 확대/중단/개선안을 제시할 수 있는가?
- 채널/세그먼트별로 실행 가능한 액션 3가지를 제안할 수 있는가?

## 2) Why this matters (Theory)
- 캡스톤은 '정답'보다 '의사결정 문서'가 목표
- 전환/매출/마진을 동시에 본다 (한 지표만 보면 실패)
- 액션은 범위(누구/어디/언제)와 리스크(가드레일)를 포함

## 3) Setup & Data Load

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf

DATA_PATH = "data/sales_marketing_example_2025.csv"
df = pd.read_csv(DATA_PATH, parse_dates=["date"])

# Quick sanity checks
display(df.head())
print("rows, cols:", df.shape)
print("missing rate (top):")
display(df.isna().mean().sort_values(ascending=False).head(10))

## 4) Helper Functions

In [ ]:
def one_sentence_insight(prompt: str):
    """Write your one-sentence business insight below."""
    print("\n🧾 One-sentence insight (fill this):")
    print(" -", prompt)

def bootstrap_mean_diff(x, y, iters=5000, seed=0):
    """Bootstrap CI for difference in means (x - y)."""
    r = np.random.default_rng(seed)
    x = np.asarray(pd.Series(x).fillna(0))
    y = np.asarray(pd.Series(y).fillna(0))
    diffs = []
    for _ in range(iters):
        xs = r.choice(x, size=len(x), replace=True)
        ys = r.choice(y, size=len(y), replace=True)
        diffs.append(xs.mean() - ys.mean())
    diffs = np.array(diffs)
    return diffs.mean(), np.quantile(diffs, [0.025, 0.975])

def rate_ci_wilson(successes, n, alpha=0.05):
    """Wilson score interval for a proportion."""
    if n == 0:
        return (np.nan, np.nan)
    z = stats.norm.ppf(1 - alpha/2)
    phat = successes / n
    denom = 1 + z**2/n
    center = (phat + z**2/(2*n)) / denom
    half = z * np.sqrt((phat*(1-phat) + z**2/(4*n)) / n) / denom
    return center - half, center + half

---
## 5) Analysis

### 5.1 Define scope & baseline
✅ 대상 시장/채널을 고르고 baseline을 정의합니다.

In [ ]:
SCOPE_COUNTRY = 'KR'
SCOPE_CHANNEL = 'Ecommerce'

sc = df[(df['country']==SCOPE_COUNTRY) & (df['channel']==SCOPE_CHANNEL)].copy()
print("scope rows:", len(sc))
print("baseline conv:", sc[sc['campaign']=='None']['conversions'].mean())
print("baseline daily revenue:", sc[sc['campaign']=='None']['revenue'].fillna(0).mean())

In [ ]:
one_sentence_insight('이 스코프를 고른 이유(비즈니스 관점) 한 문장')

### 5.2 Campaign comparison table (A/B/C/None)
✅ 전환/매출/마진을 동시에 비교합니다.

In [ ]:
def campaign_summary(d):
    d = d.copy()
    d['revenue'] = d['revenue'].fillna(0)
    d['gross_margin'] = d['gross_margin'].fillna(0)
    return (d.groupby('campaign')
              .agg(n=('campaign','size'),
                   conv_rate=('conversions','mean'),
                   rev_mean=('revenue','mean'),
                   margin_mean=('gross_margin','mean'),
                   spend_mean=('marketing_spend','mean'))
              .sort_values('conv_rate', ascending=False))

display(campaign_summary(sc))

In [ ]:
one_sentence_insight('어떤 캠페인을 확대/중단할지(가드레일 포함) 한 문장')

### 5.3 Recommendation: write 3 actions + 3 risks
✅ 최종 산출물: 액션 3개 + 리스크 3개

In [ ]:
actions = [
    "Action 1: (e.g., Expand Campaign A to KR+Ecommerce for 2 more weeks)",
    "Action 2: (e.g., Reduce discount for channel X and monitor margin guardrail)",
    "Action 3: (e.g., Reallocate spend from C to A for SMB segment)"
]

risks = [
    "Risk 1: (e.g., margin deterioration if discount increases)",
    "Risk 2: (e.g., selection bias / seasonality confound)",
    "Risk 3: (e.g., operational constraints / inventory / sales capacity)"
]

print("=== Actions ===")
for a in actions: print("-", a)
print("\n=== Risks ===")
for r in risks: print("-", r)

In [ ]:
one_sentence_insight("액션 3개를 '누구/어디/언제/성공 기준'까지 포함해 한 문장")

---
## 6) Interpretation Checklist
- 결과가 '유의'하면, **효과 크기/CI**가 실무적으로 의미 있는가?
- 결과가 '비유의'하면, **표본 부족/측정오류/세그먼트 혼합** 가능성은?
- 액션을 한다면, **누구에게/어디에/얼마나** 적용할 것인가?



## Executive Summary (메일/슬라이드에 그대로 붙여넣기)

- **핵심 질문:**  
- **결론(한 문장):**  
- **근거 지표:** (효과크기 / p-value / CI 등)  
- **비즈니스 임팩트:** (예: 월 매출 +X, 마진 -Y 위험)  
- **권장 액션:** (실험/타겟 변경/채널 믹스 조정)  
- **추가 분석 필요:**  
